<a href="https://colab.research.google.com/github/ayulockin/examples/blob/master/colabs/integrations/mmdetection/Train_Object_Detector_with_MMDetection_and_W%26B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="http://wandb.me/logo-im-png" width="400" alt="Weights & Biases"/> <br>

<!--- @wandbcode{mmdetection-wandb-colab, v=1} -->

<img src="http://wandb.me/mini-diagram" width="600" alt="Weights & Biases"/>

# 💡 Train an Object Detector with MMDetection and Weights and Biases

In this colab, we will train an object detector using [MMDetection](https://mmdetection.readthedocs.io/en/latest/1_exist_data_model.html) on a tiny [Kitti](https://paperswithcode.com/dataset/kitti) dataset. Through this colab you will learn to:

* use MMDetection to train an object detector on a custom dataset,
* use [Weights and Biases](https://wandb.ai/site) to log training and validation metrics, visualize model predictions, version raw validation dataset, and more.

This colab in particular, will showcase a dedicated `MMDetWandbHook` for MMDetection that can be used to:

✅ Log training and evaluation metrics. <br>
✅ Log versioned model checkpoints. <br>
✅ Log versioned validation dataset with ground truth bounding boxes. <br>
✅ Log and visualize model predictions.

But before we continue, here's a quick summary of MMDetection and W&B if you are not familiar with them.

### 📸 MMDetection

MMDetection is an open source object detection toolbox based on PyTorch. It provides composable components that are easy to customize and has out-of-box support for single and multi GPU training/inference. It also has hundreds of pretrained detection models in Model Zoo, and supports multiple standard datasets. Check out the GitHub repository [here](https://github.com/open-mmlab/mmdetection).

### 📸 Weights and Biases

Consider **[Weights and Biases](https://wandb.ai/site)** (W&B) to be the GitHub for machine learning. Use W&B for machine learning experiment tracking, dataset and model versioning, project collaboration, hyperparameter optimization, dataset exploration, model evaluation and so much more. If you are new to W&B, check out this [intro colab](https://wandb.me/intro).

# ⚽️ Imports and Setup

### 1️⃣ Install MMDetection

MMDetection is heavily dependent on the [MMCV](https://mmcv.readthedocs.io/en/latest/#installation) library. We will have to install the version of MMCV that is compatible with the given PyTorch version. Check out the [Installation documentation](https://mmdetection.readthedocs.io/en/latest/get_started.html#installation) for more details. 

In [ ]:
# install dependencies: (use cu111 because colab has CUDA 11.1)
!pip install -qq torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
!pip install -qq mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html

# Install mmdetection
!rm -rf mmdetection
!git clone -b wandb2 https://github.com/ayulockin/mmdetection/
%cd mmdetection

!pip install -e .

     |█████████████                   | 834.1 MB 1.6 MB/s eta 0:12:51tcmalloc: large alloc 1147494400 bytes == 0x557640bc0000 @  0x7f0d2e163615 0x557607f893bc 0x55760806a18a 0x557607f8c1cd 0x55760807eb3d 0x557608000458 0x557607ffb02f 0x557607f8daba 0x5576080002c0 0x557607ffb02f 0x557607f8daba 0x557607ffccd4 0x55760807f986 0x557607ffc350 0x55760807f986 0x557607ffc350 0x55760807f986 0x557607ffc350 0x557607f8df19 0x557607fd1a79 0x557607f8cb32 0x5576080001dd 0x557607ffb02f 0x557607f8daba 0x557607ffccd4 0x557607ffb02f 0x557607f8daba 0x557607ffbeae 0x557607f8d9da 0x557607ffc108 0x557607ffb02f
     |████████████████▌               | 1055.7 MB 1.5 MB/s eta 0:11:03tcmalloc: large alloc 1434370048 bytes == 0x557685216000 @  0x7f0d2e163615 0x557607f893bc 0x55760806a18a 0x557607f8c1cd 0x55760807eb3d 0x557608000458 0x557607ffb02f 0x557607f8daba 0x5576080002c0 0x557607ffb02f 0x557607f8daba 0x557607ffccd4 0x55760807f986 0x557607ffc350 0x55760807f986 0x557607ffc350 0x55760807f986 0x557607ffc350 0x5576

### 2️⃣ Install Weights and Biases

Install the latest version of W&B. 

In [ ]:
!pip install -q --upgrade wandb

     |████████████████████████████████| 1.7 MB 5.3 MB/s 
     |████████████████████████████████| 181 kB 47.7 MB/s 
     |████████████████████████████████| 144 kB 46.3 MB/s 
     |████████████████████████████████| 63 kB 1.3 MB/s 


### 3️⃣ General Imports

In [ ]:
import os
import os.path as osp
import torch
import torchvision
import numpy as np

# MMDetection
import mmdet
print(mmdet.__version__)
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset
from mmdet.apis import set_random_seed

# MMCV
import mmcv
from mmcv import Config

# Weights and Biases
import wandb
print(wandb.__version__)

2.22.0
0.12.11


### 4️⃣ Login with you W&B account

Create a free W&B account (it's free for personal and academic usage). Visit wandb.ai/authorize to get your unique authentication token. 

In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# 🏀 Dataset

We will be using a tiny KITTI dataset for this colab notebook. 

Even though KITTI is a standard dataset for object detection, tiny KITTI can be considered as a custom dataset (lesser number of classes). MMDetection, recommends to convert the data into COCO or PASCAL VOC formats or the middle format.

### 1️⃣ Download the dataset

In [ ]:
%cd ../
!wget https://download.openmmlab.com/mmdetection/data/kitti_tiny.zip
!unzip -q kitti_tiny.zip

/content
--2022-03-24 08:51:00--  https://download.openmmlab.com/mmdetection/data/kitti_tiny.zip
Resolving download.openmmlab.com (download.openmmlab.com)... 47.252.96.28
Connecting to download.openmmlab.com (download.openmmlab.com)|47.252.96.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6918271 (6.6M) [application/zip]
Saving to: ‘kitti_tiny.zip’

kitti_tiny.zip      100%[===================>]   6.60M  8.35MB/s    in 0.8s    

2022-03-24 08:51:02 (8.35 MB/s) - ‘kitti_tiny.zip’ saved [6918271/6918271]



In [ ]:
!ls kitti_tiny

training  train.txt  val.txt


> Note: The `training` folder contains both training and validation data samples. This split is determined by the `train.txt` and `val.txt` files. 

### 2️⃣ Build Custom Dataloader

To support a new data format, it's recommended to convert the annotations to COCO format or PASCAL VOC format. You can also convert them the "middle format". 

If you are converting annotations to COCO format, do so offline and use the `CocoDataset` class. If you are converting it to the PASCAL format, use the `VOCDataset` class.

In the example below, we are converting it to the middle format. The `KittiTinyDataset` class will thus inherit the `CustomDataset` class and override the `load_annotations` method. 

You can find more details about customizing the dataset [here](https://mmdetection.readthedocs.io/en/latest/tutorials/customize_dataset.html).

In [ ]:
@DATASETS.register_module()
class KittiTinyDataset(CustomDataset):

    CLASSES = ('Car', 'Pedestrian', 'Cyclist')

    def load_annotations(self, ann_file):
        cat2label = {k: i for i, k in enumerate(self.CLASSES)}
        # load image list from file
        image_list = mmcv.list_from_file(self.ann_file)
    
        data_infos = []
        # convert annotations to middle format
        for image_id in image_list:
            filename = f'{self.img_prefix}/{image_id}.jpeg'
            image = mmcv.imread(filename)
            height, width = image.shape[:2]
    
            data_info = dict(filename=f'{image_id}.jpeg', width=width, height=height)
    
            # load annotations
            label_prefix = self.img_prefix.replace('image_2', 'label_2')
            lines = mmcv.list_from_file(os.path.join(label_prefix, f'{image_id}.txt'))
    
            content = [line.strip().split(' ') for line in lines]
            bbox_names = [x[0] for x in content]
            bboxes = [[float(info) for info in x[4:8]] for x in content]
    
            gt_bboxes = []
            gt_labels = []
            gt_bboxes_ignore = []
            gt_labels_ignore = []
    
            # filter 'DontCare'
            for bbox_name, bbox in zip(bbox_names, bboxes):
                if bbox_name in cat2label:
                    gt_labels.append(cat2label[bbox_name])
                    gt_bboxes.append(bbox)
                else:
                    gt_labels_ignore.append(-1)
                    gt_bboxes_ignore.append(bbox)

            data_anno = dict(
                bboxes=np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4),
                labels=np.array(gt_labels, dtype=np.long),
                bboxes_ignore=np.array(gt_bboxes_ignore,
                                       dtype=np.float32).reshape(-1, 4),
                labels_ignore=np.array(gt_labels_ignore, dtype=np.long))

            data_info.update(ann=data_anno)
            data_infos.append(data_info)

        return data_infos

# 🏈 Model

There are over hundred pre-trained object detectors provided by MMDetection via Model Zoo. Check out the Model Zoo [documentation](https://mmdetection.readthedocs.io/en/v2.21.0/model_zoo.html) page.

You can also customize the model's backbone, neck, head, ROI, and loss. More on customizing the model [here](https://mmdetection.readthedocs.io/en/latest/tutorials/customize_models.html).

### 1️⃣ Download the model

We will be using a pretrained model checkpoint to fine tune on our custom dataset. Let's download the model in the `checkpoints` directory.

In [ ]:
!mkdir checkpoints
!wget -c https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_caffe_fpn_mstrain_3x_coco/faster_rcnn_r50_caffe_fpn_mstrain_3x_coco_20210526_095054-1f77628b.pth \
      -O checkpoints/faster_rcnn_r50_caffe_fpn_mstrain_3x_coco_20210526_095054-1f77628b.pth

--2022-03-24 08:51:03--  https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_caffe_fpn_mstrain_3x_coco/faster_rcnn_r50_caffe_fpn_mstrain_3x_coco_20210526_095054-1f77628b.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.252.96.28
Connecting to download.openmmlab.com (download.openmmlab.com)|47.252.96.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 167291982 (160M) [application/octet-stream]
Saving to: ‘checkpoints/faster_rcnn_r50_caffe_fpn_mstrain_3x_coco_20210526_095054-1f77628b.pth’

checkpoints/faster_ 100%[===================>] 159.54M  8.50MB/s    in 19s     

2022-03-24 08:51:22 (8.50 MB/s) - ‘checkpoints/faster_rcnn_r50_caffe_fpn_mstrain_3x_coco_20210526_095054-1f77628b.pth’ saved [167291982/167291982]



# ⚾️ Configuration

MMDetection relies heavily on a config system. In the cell below, we will be loading a config file and modify few of the methods as per the need of this notebook.

Note that both train and test dataloaders will use the same training samples. This is not a recommended practice but for the sake of a simplified notebook, let's use it. 

Learn more about the MMDetection Config system [here](https://mmdetection.readthedocs.io/en/latest/tutorials/config.html).

### 1️⃣ Load the config file

In [ ]:
cfg = Config.fromfile('mmdetection/configs/faster_rcnn/faster_rcnn_r50_caffe_fpn_mstrain_1x_coco.py')

### 2️⃣ Modify data config

In [ ]:
# Define type and path to the images.
cfg.dataset_type = 'KittiTinyDataset'
cfg.data_root = 'kitti_tiny/'

cfg.data.test.type = 'KittiTinyDataset'
cfg.data.test.data_root = 'kitti_tiny/'
cfg.data.test.ann_file = 'train.txt'
cfg.data.test.img_prefix = 'training/image_2'

cfg.data.train.type = 'KittiTinyDataset'
cfg.data.train.data_root = 'kitti_tiny/'
cfg.data.train.ann_file = 'train.txt'
cfg.data.train.img_prefix = 'training/image_2'

cfg.data.val.type = 'KittiTinyDataset'
cfg.data.val.data_root = 'kitti_tiny/'
cfg.data.val.ann_file = 'val.txt'
cfg.data.val.img_prefix = 'training/image_2'

### 3️⃣ Modify model config

In [ ]:
# The number of unique objects in the training data.
cfg.model.roi_head.bbox_head.num_classes = 3
# Use the pretrained model.
cfg.load_from = 'checkpoints/faster_rcnn_r50_caffe_fpn_mstrain_3x_coco_20210526_095054-1f77628b.pth'

### 4️⃣ Modify training config

In [ ]:
# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer.lr = 0.02 / 8
cfg.lr_config.warmup = None
cfg.log_config.interval = 10

# Epochs
cfg.runner.max_epochs = 12

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

# ⭐️ Set the checkpoint interval.
cfg.checkpoint_config.interval = 1

# Set up working dir to save files and logs.
cfg.work_dir = './tutorial_exps'

### 5️⃣ Modify evaluation config

In [ ]:
# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = 'mAP'

# ⭐️ Set the evaluation interval.
cfg.evaluation.interval = 1

# 🎾 Define Weights and Biases Hook

MMDetection comes with a dedicated Weights and Biases Hook - `MMDetWandHook`. MMCV, the parent repository, has a `WandbLoggerHook` that can be used to for basic logging. 

With this dedicated hook, you can:

* log train and eval metrics along with system (CPU/GPU) metrics, 
* visualize the validation dataset as interactive [W&B Tables](https://docs.wandb.ai/guides/data-vis),
* visualize the model prediction as interactive W&B Tables, and
* save the model checkpoints as [W&B Artifacts](https://docs.wandb.ai/guides/artifacts).

To use this hook, you can append a dict to `log_config.hooks`. The `log_config` wraps multiple logger hooks like  the `TextLoggerHook` used below.

There are four important arguments in the `MMDetWandbHook` that can help you get the most out of MMDetection. 

- `init_kwargs`: Use this argument to in-turn pass arguments to `wandb.init`. You can use it to set the W&B project name, set the team name entity if you want to log the runs to a team account, pass the configuration, and more. Check out what all can you pass to `wandb.init` [here](https://docs.wandb.ai/ref/python/init).

- `log_checkpoint`: The model checkpoints are saved at intervals determined by `checkpoint_config.interval` (starred above). If `log_checkpoint` is `True` the saved checkpoints will be saved as versioned W&B Artifact. Note that this feature is dependent on MMCV's [`CheckpointHook`](https://mmcv.readthedocs.io/en/latest/api.html#mmcv.runner.CheckpointHook).

- `log_checkpoint_metadata`: If `log_checkpoint_metadata` is True, every checkpoint artifact will have a metadata associated with it. The metadata contains the evaluation metrics computed on validation data with that checkpoint along with the current epoch. If True, it also marks the checkpoint version with the best evaluation metric with a `best` alias. You can choose the best checkpoint in the W&B Artifacts UI using this.

- `num_eval_images`: At every evaluation interval, the `MMDetWandbHook` logs the model prediction as interactive W&B Tables. The eval interval is determined by `evaluation.interval` (starred above). The number of samples logged is given by `num_eval_images`. The predicted bounding boxes along with the ground truth are logged at every evaluation interval. However, the validation data is logged just once. This Feature is dependent on MMCV's [`EvalHook`](https://mmcv.readthedocs.io/en/latest/api.html#mmcv.runner.EvalHook) or [`DistEvalHook`](https://mmcv.readthedocs.io/en/latest/api.html#mmcv.runner.DistEvalHook).

In [ ]:
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='MMDetWandbHook',
         init_kwargs={'project': 'MMDetection-tutorial'},
         interval=10,
         log_checkpoint=True,
         log_checkpoint_metadata=True,
         num_eval_images=10)]

# 🏐 Train

Now that we have the dataset, pretrained model weight, and have defined the configs. Let's stitch them together to train an object detector.

### 1️⃣ Build the Dataset

In [ ]:
# Build dataset
datasets = [build_dataset(cfg.data.train)]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int`

### 2️⃣ Build the Model

In [ ]:
# Build the detector
model = build_detector(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

### 3️⃣ Train with W&B

In [ ]:
# Create work_dir
# mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int`

/content/mmdetection/mmdet/core/hook/wandblogger_hook.py:109: UserWarning: To log mmdetection Config, pass it to init_kwargs of MMDetWandbHook.
  'pass it to init_kwargs of MMDetWandbHook.', UserWarning)
2022-03-24 08:56:00,585 - mmdet - INFO - Epoch [1][10/25]	lr: 2.500e-03, eta: 0:05:39, time: 1.171, data_time: 0.230, memory: 2770, loss_rpn_cls: 0.0250, loss_rpn_bbox: 0.0173, loss_cls: 0.5387, acc: 81.6016, loss_bbox: 0.3951, loss: 0.9761
2022-03-24 08:56:10,622 - mmdet - INFO - Epoch [1][20/25]	lr: 2.500e-03, eta: 0:05:04, time: 1.003, data_time: 0.059, memory: 2770, loss_rpn_cls: 0.0135, loss_rpn_bbox: 0.0119, loss_cls: 0.1753, acc: 93.4668, loss_bbox: 0.3242, loss: 0.5249
2022-03-24 08:56:15,244 - mmdet - INFO - Saving checkpoint at 1 epochs


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 25/25, 3.9 task/s, elapsed: 6s, ETA:     0s
---------------iou_thr: 0.5---------------


2022-03-24 08:56:24,478 - mmdet - INFO - 
+------------+-----+------+--------+-------+
| class      | gts | dets | recall | ap    |
+------------+-----+------+--------+-------+
| Car        | 62  | 691  | 0.952  | 0.764 |
| Pedestrian | 13  | 105  | 0.846  | 0.805 |
| Cyclist    | 7   | 96   | 0.571  | 0.109 |
+------------+-----+------+--------+-------+
| mAP        |     |      |        | 0.559 |
+------------+-----+------+--------+-------+
2022-03-24 08:56:24,482 - mmdet - INFO - Epoch(val) [1][25]	AP50: 0.5590, mAP: 0.5591



---------------iou_thr: 0.5---------------

---------------iou_thr: 0.5---------------


...

2022-03-24 09:03:18,372 - mmdet - INFO - Epoch [12][10/25]	lr: 2.500e-05, eta: 0:00:13, time: 1.229, data_time: 0.256, memory: 2770, loss_rpn_cls: 0.0013, loss_rpn_bbox: 0.0061, loss_cls: 0.0542, acc: 97.9395, loss_bbox: 0.1197, loss: 0.1813
2022-03-24 09:03:28,510 - mmdet - INFO - Epoch [12][20/25]	lr: 2.500e-05, eta: 0:00:04, time: 1.012, data_time: 0.060, memory: 2770, loss_rpn_cls: 0.0019, loss_rpn_bbox: 0.0048, loss_cls: 0.0484, acc: 98.0273, loss_bbox: 0.0887, loss: 0.1438
2022-03-24 09:03:33,185 - mmdet - INFO - Saving checkpoint at 12 epochs


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 25/25, 3.9 task/s, elapsed: 6s, ETA:     0s
---------------iou_thr: 0.5---------------


2022-03-24 09:03:42,359 - mmdet - INFO - 
+------------+-----+------+--------+-------+
| class      | gts | dets | recall | ap    |
+------------+-----+------+--------+-------+
| Car        | 62  | 132  | 0.952  | 0.851 |
| Pedestrian | 13  | 42   | 0.846  | 0.790 |
| Cyclist    | 7   | 52   | 0.429  | 0.081 |
+------------+-----+------+--------+-------+
| mAP        |     |      |        | 0.574 |
+------------+-----+------+--------+-------+
2022-03-24 09:03:42,363 - mmdet - INFO - Epoch(val) [12][25]	AP50: 0.5740, mAP: 0.5742



---------------iou_thr: 0.5---------------

---------------iou_thr: 0.5---------------



learning_rate,████████████████▂▂▂▂▂▂▁▁
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/acc,▁▆▇▇▇▆▇▇▇▇█▇█▇██████████
train/loss,█▄▄▃▂▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
train/loss_bbox,█▆▅▄▂▅▄▄▄▃▃▃▃▃▂▃▂▁▂▂▂▂▂▁
train/loss_cls,█▃▂▂▂▂▂▂▂▂▁▂▁▂▁▁▁▁▁▁▁▁▁▁
train/loss_rpn_bbox,█▅▆▆▄▆▆▅▄▄▃▄▄▅▃▃▃▂▃▂▂▂▂▁
train/loss_rpn_cls,█▅▆▅▂▃▂▂▂▂▂▁▂▂▂▁▁▁▂▁▁▁▁▁
val/AP50,▁█▄▅▂▆▃▂▂▃▃▃
val/mAP,▁█▄▅▂▆▃▂▂▃▃▃
val/val_step,▁▂▂▃▄▄▅▅▆▇▇█


### 4️⃣ Notes on using `MMDetWandbHook`. 

Using `MMDetWandbHook` is easy and in most cases it will throw friendly `UserWarning` if something is not quite right. However in the best interest, here are some of things and best practices you should keep in mind:

* The `MMDetWandbHook` depends on `CheckpointHook` for logging the checkpoints as W&B Artifacts and `EvalHook`/`DistEvalHook` for logging validation data and model predictions. If anyone or both aren't available, this hook will give `UserWarning` and not cause any error. 

* The priority of both `CheckpointHook` and `EvalHook`/`DistEvalHook` should be more than `MMDetWandbHook`. 

* The validation data is logged once as `val_data` W&B Table. The evaluation tables, use reference to this data thus you will not be uploading the same data multiple times. 

* If you want to log the configuration to W&B, pass this key-value pair `'config': cfg._cfg_dict.to_dict()` to `init_kwargs`. 